In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.image import imread

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
data = pd.read_csv("data/train_set.csv")
data.head()

,label,id_img,path
0,happy,22373,happy/22373.jpg
1,happy,21433,happy/21433.jpg
2,happy,12418,happy/12418.jpg
3,happy,21278,happy/21278.jpg
4,happy,8081,happy/08081.jpg


In [3]:
data.label.value_counts()

happy      3285
sadness    2891
Name: label, dtype: int64

In [4]:
_ = imread("data/train/" + data.iloc[0, 2])
_

array([[ 25,  44,  56, ...,  34,  11,  17],
       [ 30,  46,  54, ...,  45,  19,  17],
       [ 38,  50,  55, ...,  53,  25,  12],
       ...,
       [110, 116, 129, ...,  40,  63,  34],
       [110, 138, 135, ...,  51,  52,  30],
       [ 93, 142, 127, ...,  73,  48,  33]], dtype=uint8)

In [72]:
data["X"] = data.path.apply(lambda x: imread("data/train/" + x))
data["label"].replace(["happy", "sadness"], [1, 0], inplace = True)
data.head(2)

,label,id_img,path,X
0,1,22373,happy/22373.jpg,"[[25, 44, 56, 68, 88, 98, 93, 92, 105, 120, 13..."
1,1,21433,happy/21433.jpg,"[[33, 29, 22, 18, 19, 23, 22, 19, 20, 23, 14, ..."


In [73]:
data.label

0       1
1       1
2       1
3       1
4       1
       ..
6171    0
6172    0
6173    0
6174    0
6175    0
Name: label, Length: 6176, dtype: int64

In [74]:
# Dependent variables
X = np.array(list(data.iloc[:, 3].values))
X = X.reshape(6176, 48, 48, 1)
#X = np.asarray(X).astype('float32')

# Independent variable
y = np.array(data.iloc[:, 0])
#y = np.asarray(y)


print("X shape", X.shape)
print("y shape", y.shape)

X shape (6176, 48, 48, 1)
y shape (6176,)


In [76]:
model = models.Sequential([
    layers.Conv2D(filters = 8,
                  kernel_size = (3, 3),
                  input_shape = (48, 48, 1),
                  padding = "same"),
    #layers.MaxPool2D((2, 2)),
    layers.Dropout(.25),
    layers.Flatten(),
    layers.Dense(32, activation = "relu"),
    layers.Dense(2, activation = "sigmoid")
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_history = model.fit(X, y, epochs = 10)

Epoch 1/10
193/193 [==============================] - 3s 12ms/step - loss: 33.4158 - accuracy: 0.5667
Epoch 2/10
193/193 [==============================] - 2s 12ms/step - loss: 0.7520 - accuracy: 0.5050
Epoch 3/10
193/193 [==============================] - 2s 12ms/step - loss: 0.7309 - accuracy: 0.5316
Epoch 4/10
193/193 [==============================] - 2s 12ms/step - loss: 0.6613 - accuracy: 0.5826
Epoch 5/10
193/193 [==============================] - 2s 13ms/step - loss: 0.6463 - accuracy: 0.6316
Epoch 6/10
193/193 [==============================] - 3s 13ms/step - loss: 0.6276 - accuracy: 0.6546
Epoch 7/10
193/193 [==============================] - 3s 13ms/step - loss: 0.6581 - accuracy: 0.5808
Epoch 8/10
193/193 [==============================] - 2s 13ms/step - loss: 0.6171 - accuracy: 0.6575
Epoch 9/10
193/193 [==============================] - 3s 14ms/step - loss: 0.6067 - accuracy: 0.6773
Epoch 10/10
193/193 [==============================] - 3s 14ms/step - loss: 0.5922 - accur